In [1]:
from serpapi import GoogleSearch

def get_query(query):
    params = {
        "q": query,
        "engine": "google",
        "api_key": "d5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results['organic_results']
    return organic_results

Query = "Professor Kevin Chang uiuc"
organic_results = get_query(Query)



In [2]:
import trafilatura as tr
import pandas as pd
import wikipedia as wiki

def parse_results(link):
    downloaded = tr.fetch_url(link)
    text = tr.extract(downloaded, include_formatting=True)
    if text == None:
        return ''
    return text

def get_scholar_results(author_id):
    params = {
        'engine': 'google_scholar_author',
        'author_id': author_id,
        'api_key' : 'd5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f'
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

dataset = {'titles': [], 'text': []}


for result in organic_results:
    if 'wikipedia.org' in result['link']:
        with open("Documents/wiki.txt", 'w') as f:
            wiki_results = wiki.search(result['title'])
            page = wiki.page(wiki_results[0], auto_suggest=False)
            f.write(page.content)
        dataset['titles'].append("wikiresult")
        dataset['text'].append(page.content)
            
    else:
        text = parse_results(result['link'])
        with open("Documents/" + result['title'] + '.txt', 'w') as f:
            f.write(parse_results(result['link']))
        dataset['titles'].append(result['title'])
        dataset['text'].append(parse_results(result['link']))

df = pd.DataFrame(dataset)
df.head()


,titles,text
0,Kevin Chenchuan Chang - Computer Science | UIUC,# Kevin Chenchuan Chang\n## For More Informati...
1,Kevin C.C. Chang - Professor - University of I...,
2,Kevin Chen-Chuan Chang,|A comprehensive survey of graph embedding: Pr...
3,Kevin Chang at University Of Illinois at Urban...,Professors\ncancel\nat\nLog In\nSign Up\n3.5\n...
4,Kevin Chen-Chuan Chang,


In [10]:
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import numpy as np
import re

def preprocess_documents(dataframe):
    passages = []
    titles = []
    doc_idx_map = []
    passage_idx_map = []
    for doc_idx, row in dataframe.iterrows():
        title = row['titles']
        document = row['text']

        # Check if the document contains headings
        if "#" in document or "##" in document or "###" in document:
            # Split the document into sections based on the headings
            sections = re.split(r'\n#+\s+', document)[1:]  # Skip the first empty section

            for section_idx, section in enumerate(sections, 1):
                # Split each section into heading and content
                section_parts = section.split("\n\n", 1)

                # Extract the heading (if available)
                section_heading = section_parts[0].strip() if len(section_parts) > 1 else ""

                # Extract the content (if available)
                section_content = section_parts[1].strip() if len(section_parts) > 1 else section.strip()

                # Keep the entire section as a single passage
                passage_with_heading = f"{section_heading} {section_content}" if section_heading else section_content
                passages.append(passage_with_heading)
                titles.append(title)
                doc_idx_map.append(doc_idx)
                passage_idx_map.append((section_idx, 0))

        else:
            # No headings found, treat the entire document as a single passage
            if document.strip():
                passages.append(document.strip())
                titles.append(title)
                doc_idx_map.append(doc_idx)
                passage_idx_map.append((1, 0))

    return passages, titles, doc_idx_map, passage_idx_map



def encode_question_and_passages(question, passages, tokenizer):
    encoded = tokenizer(question, passages, padding=True, truncation=True, return_tensors="pt")
    return encoded

def retrieve_top_passages(question, passages, titles, doc_idx_map, passage_idx_map, model, tokenizer, top_k=5):
    encoded = encode_question_and_passages(question, passages, tokenizer)
    with torch.no_grad():
        outputs = model(**encoded)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    max_length = encoded['input_ids'].shape[1]
    passage_scores = (start_logits + end_logits) / 2  # Average the start and end logits

    # Ensure top_k does not exceed the available passages
    top_k = min(top_k, len(passage_scores[0]))


    top_passage_indices = passage_scores[0].argsort(descending=True)[:top_k].cpu().numpy()  # Convert to numpy array
    print(top_passage_indices)
    top_passages = []
    for idx in top_passage_indices:
        doc_idx = doc_idx_map[idx]
        section_idx, passage_idx = passage_idx_map[idx]
        title = titles[doc_idx]
        passage_text = passages[idx]
        top_passages.append({
            "title": title,
            "section_index": section_idx,
            "passage_index": [passage_idx],  # Convert passage_idx to a list or tuple
            "passage_text": passage_text
        })

    return top_passages



question = "Courses taught Recently"

passages, titles, doc_idx_map, passage_idx_map = preprocess_documents(df)


print(len(passages))

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Retrieve the top 5 most relevant passages for the given question
top_passages = retrieve_top_passages(question, passages, titles, doc_idx_map, passage_idx_map, model, tokenizer)

top_passages


16


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[ 5 11 15 12 10]


[{'title': 'Kevin Chenchuan Chang - Computer Science | UIUC',
  'section_index': 6,
  'passage_index': [0],
  'passage_text': 'Research Interests\n- Natural language processing, data mining, data management, and information retrieval with machine learning techniques, with emphasis on the application areas of Web and social media-based knowledge acquisition and organization across structured and unstructured data.'},
 {'title': 'Kevin Chenchuan Chang - Computer Science | UIUC',
  'section_index': 1,
  'passage_index': [0],
  'passage_text': '|A comprehensive survey of graph embedding: Problems, techniques, and applications|\nH Cai, VW Zheng, KCC Chang\nIEEE transactions on knowledge and data engineering 30 (9), 1616-1637, 2018\n|1778||2018|\n|Geom-gcn: Geometric graph convolutional networks|\nH Pei, B Wei, KCC Chang, Y Lei, B Yang\narXiv preprint arXiv:2002.05287, 2020\n|621||2020|\n|Structured databases on the web: Observations and implications|\nKCC Chang, B He, C Li, M Patel, Z Zhang